# Converter um modelo do keras para tensorflow

Ao utilizar o modulo DNN do opencv é possível que possamos rodar o modelo do tensorflow diretamente com o Opencv, contudo pra que isso seja possível, devemos respeitar algumas limitações do modulo OpenCV.
Algumas limitações conhecidas:
* Não há suporte direto para ativações LeaKY ReLU
* O modelo deve estar no formato .pb e não .h5 (keras)
* Variaveis de treinamento não devem estar incluidas no modelo final

In [1]:
import os
import time

import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# Carregar modelo e alterar seu formato
Por vezes salvar um modelo (.h5) sem otimizador conseguimos reduzir o tamanho do nosso modelo em até 3x, essas tipo de situação é mais recorrente quando utilizamos otimizadores como Adam, Adadelta, Adagrad, etc

In [2]:
MODEL_PATH = "./nose_tracker_best_loss.h5"

In [4]:
# Salva no formato keras e tensorflow
keras_model = keras.models.load_model(MODEL_PATH, compile=False)
keras_model.save(f"{os.path.splitext(MODEL_PATH)[0]}_no_opt.h5", include_optimizer=False)

frozen_graph_filename = os.path.splitext(MODEL_PATH)[0]
keras_model.save(frozen_graph_filename)

INFO:tensorflow:Assets written to: ./nose_tracker_best_loss\assets


INFO:tensorflow:Assets written to: ./nose_tracker_best_loss\assets


# Congelar o grafo e remover variaveis de treinamento

In [5]:
loaded = tf.saved_model.load(frozen_graph_filename)
infer = loaded.signatures['serving_default']

f = tf.function(infer).get_concrete_function(
    **{keras_model.input.name: tf.TensorSpec(shape=keras_model.input_shape,
                                                  dtype=tf.float32)})
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()

# Export frozen graph
with tf.io.gfile.GFile(frozen_graph_filename + ".pb", 'wb') as f:
    f.write(graph_def.SerializeToString())

# Comparar predições Keras vs Opencv

In [6]:
random_input = np.random.rand(*((1,) + tuple(keras_model.input.shape[1:]))).astype(np.float32)
pred_keras = keras_model.predict(random_input)

random_input = np.transpose(random_input, [0, 3, 1, 2])
model_dnn = cv2.dnn.readNetFromTensorflow(f"{frozen_graph_filename}.pb")
model_dnn.setInput(random_input)
pred_opencv = model_dnn.forward()

diff = abs(pred_keras - pred_opencv).sum()
print(f"\nPrediction diff between opencv vs keras = {diff}")


1/1 [==============================] - 1s 880ms/step


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\net.cpp:79: error: (-215:Assertion failed) !empty() in function 'cv::dnn::dnn4_v20220524::Net::forward'


# Comaparar velocidade Keras vs Opencv

In [ ]:
started_time = time.time()
for _ in range(100):
    random_input = np.random.rand(*((1,) + tuple(keras_model.input.shape[1:]))).astype(np.float32)
    pred_keras = keras_model.predict(random_input)
print(time.time() - started_time)

started_time = time.time()
for _ in range(100):
    random_input = np.random.rand(*((1,) + tuple(keras_model.input.shape[1:]))).astype(np.float32)
    random_input = np.transpose(random_input, [0, 3, 1, 2])
    model_dnn.setInput(random_input)
    pred_opencv = model_dnn.forward()
print(time.time() - started_time)

# Inferencia com Opencv

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        raise Exception("Ooops...")
    batch = cv2.resize(frame, (112, 112))
    batch = batch / 255
    batch = batch[np.newaxis]
    batch = np.transpose(batch, [0, 3, 1, 2])
    model_dnn.setInput(batch)
    x, y = model_dnn.forward()[0]
    x = int(x*frame.shape[1])
    y = int(y*frame.shape[0])
    cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)
    cv2.imshow("debug", frame)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cv2.destroyAllWindows()
cap.release()